### Now, we will write a Python function to:

Standardize column names
Remove duplicate rows
Clean messy text values
Handle missing values (even disguised ones!)
Flag weird stuff like constant columns and outliers
Prepare your data for modelling by converting text columns smartly
Here’s how to write such a function:

In [5]:
import pandas as pd
df = pd.read_csv('loan-recovery.csv')
print("Original Data Info:")
print(df.info())

Original Data Info:
<class 'pandas.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Borrower_ID              500 non-null    str    
 1   Age                      500 non-null    int64  
 2   Gender                   500 non-null    str    
 3   Employment_Type          500 non-null    str    
 4   Monthly_Income           500 non-null    int64  
 5   Num_Dependents           500 non-null    int64  
 6   Loan_ID                  500 non-null    str    
 7   Loan_Amount              500 non-null    int64  
 8   Loan_Tenure              500 non-null    int64  
 9   Interest_Rate            500 non-null    float64
 10  Loan_Type                500 non-null    str    
 11  Collateral_Value         500 non-null    float64
 12  Outstanding_Loan_Amount  500 non-null    float64
 13  Monthly_EMI              500 non-null    float64
 14  Payment_History  

In [6]:
import pandas as pd
import numpy as np

def clean_dataframe(df: pd.DataFrame, verbose: bool = True) -> pd.DataFrame:
    df = df.copy()

    # log helper
    def log(msg):
        if verbose:
            print(f"[INFO] {msg}")

    # 1. standardize column names
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
    log("Standardized column names.")

    # 2. remove exact duplicates
    dup_count = df.duplicated().sum()
    if dup_count > 0:
        df.drop_duplicates(inplace=True)
        log(f"Removed {dup_count} duplicate rows.")

    # 3. trim and lowercase all string (object) values
    for col in df.select_dtypes(include='object'):
        df[col] = df[col].astype(str).str.strip().str.lower()
    log("Standardized string columns (lowercase + trimmed).")

    # 4. detect missing values (including blanks and placeholders)
    placeholder_values = ['n/a', 'na', '--', '-', 'none', 'null', '', 'nan']
    df.replace(placeholder_values, np.nan, inplace=True)
    null_report = df.isnull().sum()
    null_report = null_report[null_report > 0]
    if not null_report.empty:
        log(f"Missing values found in columns:\n{null_report}")

    # 5. flag constant columns
    constant_cols = [col for col in df.columns if df[col].nunique() == 1]
    if constant_cols:
        log(f"Constant columns (consider removing): {constant_cols}")

    # 6. flag high cardinality categorical columns
    high_card_cols = [col for col in df.select_dtypes(include='object') if df[col].nunique() > 100]
    if high_card_cols:
        log(f"High-cardinality columns (consider encoding strategies): {high_card_cols}")

    # 7. detect numeric outliers using IQR
    num_cols = df.select_dtypes(include=np.number).columns
    outlier_report = {}
    for col in num_cols:
        q1, q3 = df[col].quantile([0.25, 0.75])
        iqr = q3 - q1
        lower = q1 - 1.5 * iqr
        upper = q3 + 1.5 * iqr
        outliers = df[(df[col] < lower) | (df[col] > upper)][col].count()
        if outliers > 0:
            outlier_report[col] = outliers
    if outlier_report:
        log(f"Potential numeric outliers detected:\n{outlier_report}")

    # 8. convert applicable columns to category
    for col in df.select_dtypes(include='object'):
        n_unique = df[col].nunique()
        if n_unique < len(df) * 0.05:
            df[col] = df[col].astype('category')
    log("Converted suitable object columns to category dtype.")

    log("Data cleaning complete.")
    return df

### Here’s what this Python function does:

Copies the data to keep your original safe.
Standardizes column names (lowercase, no spaces).
Removes duplicate rows if any.
Cleans string columns (trims and lowercases all text).
Handles missing values (turns things like –, N/A into NaN).
Reports missing values in each column.
Flags constant columns (same value in all rows).
Identifies high-cardinality text columns (too many unique values).
Detects numeric outliers using the IQR method.
Converts low-unique text columns to category type.
Logs every step if verbose=True.

In [7]:
import pandas as pd
df = pd.read_csv('loan-recovery.csv')
clean_df = clean_dataframe(df)
print(clean_df.info())

[INFO] Standardized column names.
[INFO] Standardized string columns (lowercase + trimmed).
[INFO] High-cardinality columns (consider encoding strategies): ['borrower_id', 'loan_id']
[INFO] Potential numeric outliers detected:
{'outstanding_loan_amount': np.int64(3), 'monthly_emi': np.int64(35), 'num_missed_payments': np.int64(21), 'collection_attempts': np.int64(40)}
[INFO] Converted suitable object columns to category dtype.
[INFO] Data cleaning complete.
<class 'pandas.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   borrower_id              500 non-null    str     
 1   age                      500 non-null    int64   
 2   gender                   500 non-null    category
 3   employment_type          500 non-null    category
 4   monthly_income           500 non-null    int64   
 5   num_dependents           500 non-null    int64   
 6  

C:\Users\madna\AppData\Local\Temp\ipykernel_7704\3220206765.py:23: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  for col in df.select_dtypes(include='object'):
C:\Users\madna\AppData\Local\Temp\ipykernel_7704\3220206765.py:41: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migratio